In [1]:
from collections import defaultdict
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import sys
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import nltk.stem as stemmer
from nltk.corpus import stopwords,words
#from nltk.corpus import words 
#from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from textblob import TextBlob
#from num2words import num2words
from collections import defaultdict
import re
lemmatizer = WordNetLemmatizer()
from datetime import datetime
import datefinder
import nltk.tokenize as nt
from operator import itemgetter

In [2]:
IDs_marker = re.compile('\.I ')

In [3]:

def get_data(path, marker):
    with open (path,'r') as f:
        text = f.read().replace('\n',' ')
        lines = re.split(marker,text)
        lines.pop(0)
        return lines

In [4]:
corpus_data=get_data('/Users/Ahmad98/Desktop/IR_proj/cacm/cacm.all',IDs_marker)

query_data=get_data('/Users/Ahmad98/Desktop/IR_proj/cacm/query.text',IDs_marker)



In [5]:
#for i,x in enumerate(corpus_data):
#    completeName = os.path.join('/Users/Ahmad98/Desktop/IR_proj/cacm/dataset1',str(i+1)+'.text')
#    file1 = open(completeName, "w")
#    file1.write(x)
#    file1.close()


In [6]:
# process the text file
 
corpus_chunk_title = re.compile('\.[T] ')
corpus_chunk_txt = re.compile(' \.W ') # not enough
corpus_chunk_txt_pub = re.compile(' \.[W,B] ')
corpus_chunk_publication = re.compile(' \.[B] ')
corpus_chunk_author = re.compile(' \.[A] ', re.MULTILINE)
corpus_chunk_author_add_cross = re.compile(' \.[A,N,X] ',re.MULTILINE) # not enough
corpus_chunk_add_cross = re.compile(' \.[B,N,X] ')
corpus_chunk_add_cross1 = re.compile(' \.[N,X] ')

 
# process the document data
 
chunked_corpus = defaultdict(dict)
 
for line in corpus_data:
    entries= re.split(corpus_chunk_title,line)
    id = entries[0].strip() #save id
    no_id = entries[1]

 
    if len(re.split(corpus_chunk_txt, no_id)) == 2: # is there text?
        no_id_entries = re.split(corpus_chunk_txt_pub, no_id)
        chunked_corpus[id]['text'] = no_id_entries[0].strip() # save title
        chunked_corpus[id]['text'] =  chunked_corpus[id]['text']+' '+no_id_entries[1].strip() # save text
        no_title_txt = no_id_entries[2]

 
        if len(re.split(corpus_chunk_author, no_title_txt)) == 2: # is there an author?
            no_title_entries = re.split(corpus_chunk_author_add_cross, no_title_txt)
            chunked_corpus[id]['text'] =  chunked_corpus[id]['text']+' '+no_title_entries[0].strip() # save publication date
            chunked_corpus[id]['text'] =  chunked_corpus[id]['text']+' '+no_title_entries[1].strip() # save author
            chunked_corpus[id]['add_date'] = no_title_entries[2].strip() # save add date
            chunked_corpus[id]['cross-references'] = no_title_entries[3].strip() # save cross-references
 
        else:
            no_title_entries = re.split(corpus_chunk_add_cross1, no_title_txt)
            chunked_corpus[id]['text'] = chunked_corpus[id]['text']+' '+ no_title_entries[0].strip() # save publication date
            chunked_corpus[id]['author'] = ''# save author
            chunked_corpus[id]['add_date'] = no_title_entries[1].strip() # save add date
            chunked_corpus[id]['cross-references'] = no_title_entries[2].strip() # save cross-references
 
    else:
        no_id_entries = re.split(corpus_chunk_publication, no_id,1)
        chunked_corpus[id]['text'] = no_id_entries[0].strip() # save title
        no_title = no_id_entries[1]
 
        if len(re.split(corpus_chunk_author, no_title,1)) == 2: # is there an author?
            no_title_entries = re.split(corpus_chunk_author_add_cross, no_title)
            chunked_corpus[id]['text'] = chunked_corpus[id]['text']+' '+ no_title_entries[0].strip() # save publication date
            chunked_corpus[id]['text'] =  chunked_corpus[id]['text']+' '+no_title_entries[1].strip() # save author
            chunked_corpus[id]['add_date'] = no_title_entries[2].strip() # save add date
            chunked_corpus[id]['cross-references'] = no_title_entries[3].strip() # save cross-references
 
        else:
            no_title_entries = re.split(corpus_chunk_add_cross1, no_title)
            chunked_corpus[id]['text'] =  chunked_corpus[id]['text']+' '+no_title_entries[0].strip() # save publication date
            chunked_corpus[id]['author'] = '' # save author
            chunked_corpus[id]['add_date'] = no_title_entries[1].strip() # save add date
            chunked_corpus[id]['cross-references'] = no_title_entries[2].strip() # save cross-references



In [7]:
chunked_corpus

defaultdict(dict,
            {'1': {'text': 'Preliminary Report-International Algebraic Language CACM December, 1958 Perlis, A. J. Samelson,K.',
              'add_date': 'CA581203 JB March 22, 1978  8:28 PM',
              'cross-references': '100\t5\t1 123\t5\t1 164\t5\t1 1\t5\t1 1\t5\t1 1\t5\t1 205\t5\t1 210\t5\t1 214\t5\t1 1982\t5\t1 398\t5\t1 642\t5\t1 669\t5\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 1\t6\t1 165\t6\t1 196\t6\t1 196\t6\t1 1273\t6\t1 1883\t6\t1 324\t6\t1 43\t6\t1 53\t6\t1 91\t6\t1 410\t6\t1 3184\t6\t1'},
             '2': {'text': 'Extraction of Roots by Repeated Subtractions for Digital Computers CACM December, 1958 Sugai, I.',
              'add_date': 'CA581202 JB March 22, 1978  8:29 PM',
              'cross-references': '2\t5\t2 2\t5\t2 2\t5\t2'},
             '3': {'text': 'Techniques Department on Matrix Program Schemes CACM December, 1958 Friedman, M. D.',
              'add_date': 'CA581201 JB March 22, 1978  8:30 PM',
   

In [8]:
from collections import defaultdict
import re
corpus_chunk_txt =re.compile(' \.W ') 
corpus_chunk_txt_add_data = re.compile(' \.[W,N] ') # not enough
corpus_chunk_author = re.compile(' \.[A] ', re.MULTILINE)
corpus_chunk_auth_add_data = re.compile(' \.[A,N] ',re.MULTILINE) # not enough
corpus_chunk_add_data = re.compile(' \.N ') # not enoug

chunked_query = defaultdict(dict)
 
for line in query_data:
    entries= re.split(corpus_chunk_txt,line)
    id = entries[0].strip() #save id
    no_text = entries[1]
    print(no_text)
 
    if len(re.split(corpus_chunk_txt_add_data, no_text)) == 2: 
        add_text_data= re.split(corpus_chunk_auth_add_data, no_text)
        chunked_query[id]['text'] = add_text_data[0].strip() 
        chunked_query[id]['add_data'] = add_text_data[1].strip()
       
    else:
        chunked_query[id]['text'] = ''
        chunked_query[id]['add_data'] = add_text_data[0].strip()   
 
    
    


 What articles exist which deal with TSS (Time Sharing System), an operating system for IBM computers? .N  1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)   
 I am interested in articles written either by Prieve or Udo Pooch .A Prieve, B. Pooch, U. .N  2. Richard Alexander, Comp Serv, Langmuir Lab (author = Pooch or Prieve)   
 Intermediate languages used in construction of multi-targeted compilers; TCOLL .N  3. Donna Bergmark, Comp Serv, Uris Hall (intermed lang)   
 I'm interested in mechanisms for communicating between disjoint processes, possibly, but not exclusively, in a distributed environment.  I would rather see descriptions of complete mechanisms, with or without implementations, as opposed to theoretical work on the abstract problem.  Remote procedure calls and message-passing are examples of my interests. .N  4. Pavel Curtis (comm mech for disjoint processes)  
 I'd like papers on design and implementation of editing interfaces, window-managers, command interpreters, et

In [9]:
dictionary = {
    "u.s": "united states",
    "u.s.a": "united states",
    "u.n": "united nations",
    "i.e": "example",
    "e.g.": "for example",
    "m.p": "member of the house of lords",
    "IBM": "International Business Machines Corporation",
    "TSS": "Time Sharing System",
}

In [10]:
for store,x in chunked_query.items():
    text= x['text']
    for key in dictionary.keys():
       # print(dictionary[key])
        text = text.replace(key, dictionary[key])
        x['text']=text
chunked_query

defaultdict(dict,
            {'1': {'text': 'What articles exist which deal with Time Sharing System (Time Sharing System), an operating system for International Business Machines Corporation computers?',
              'add_data': '1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)'},
             '2': {'text': 'I am interested in articles written either by Prieve or Udo Pooch',
              'add_data': 'Prieve, B. Pooch, U.'},
             '3': {'text': 'Intermediate languages used in construction of multi-targeted compilers; TCOLL',
              'add_data': '3. Donna Bergmark, Comp Serv, Uris Hall (intermed lang)'},
             '4': {'text': "I'm interested in mechanisms for communicating between disjoint processes, possibly, but not exclusively, in a distributed environment.  I would rather see descriptions of complete mechanisms, with or without implementations, as opposed to theoretical work on the abstract problem.  Remote procedure calls and message-passing are examples of 

In [11]:
#from nltk.corpus import stopwords 
import string
from nltk.tokenize import word_tokenize
corpus_dict={}
file = open("/Users/Ahmad98/Desktop/IR_Proj/cacm/common_words", "r")
fileData = file.read()
file.close()
stopwords = re.findall("\S+", fileData)
for store,x in chunked_corpus.items():
    tokens=word_tokenize(x['text'])
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    stop_words = set(stopwords)
    words = [w for w in stripped if not w in stop_words]
    corpus_dict[store]=words
    



In [12]:
#Queries Text Normalizing

from nltk.corpus import stopwords 
import string
from nltk.tokenize import word_tokenize
queries_dict={}
# read stop word file
file = open("/Users/Ahmad98/Desktop/IR_Proj/cacm/common_words", "r")
fileData = file.read()
file.close()
stopwords = re.findall("\S+", fileData)
for store,x in chunked_query.items():
    tokens=word_tokenize(x['text'])
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    common_words = set(stopwords)
    words = [w for w in stripped if not w in stop_words]
    queries_dict[store]=words
corpus_dict 

{'1': ['preliminary',
  'reportinternational',
  'algebraic',
  'language',
  'cacm',
  'december',
  '',
  '1958',
  'perlis',
  '',
  'samelson',
  '',
  ''],
 '2': ['extraction',
  'roots',
  'repeated',
  'subtractions',
  'digital',
  'computers',
  'cacm',
  'december',
  '',
  '1958',
  'sugai',
  '',
  ''],
 '3': ['techniques',
  'department',
  'matrix',
  'program',
  'schemes',
  'cacm',
  'december',
  '',
  '1958',
  'friedman',
  '',
  ''],
 '4': ['glossary',
  'computer',
  'engineering',
  'programming',
  'terminology',
  'cacm',
  'november',
  '',
  '1958'],
 '5': ['squareroot',
  'approximations',
  'cacm',
  'november',
  '',
  '1958',
  'wadey',
  '',
  ''],
 '6': ['computers',
  'inspection',
  'procedures',
  'cacm',
  'november',
  '',
  '1958',
  'muller',
  '',
  ''],
 '7': ['glossary',
  'computer',
  'engineering',
  'programming',
  'terminology',
  'cacm',
  'october',
  '',
  '1958'],
 '8': ['equivalence',
  'transformation',
  'program',
  'schemes',
  

In [13]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stemm_list=[]

for store,x in chunked_corpus.items():
    for i in corpus_dict[store]:
        stemm_list.append(ps.stem(i))
    corpus_dict[store]=stemm_list
    stemm_list=[]
corpus_dict      

{'1': ['preliminari',
  'reportintern',
  'algebra',
  'languag',
  'cacm',
  'decemb',
  '',
  '1958',
  'perli',
  '',
  'samelson',
  '',
  ''],
 '2': ['extract',
  'root',
  'repeat',
  'subtract',
  'digit',
  'comput',
  'cacm',
  'decemb',
  '',
  '1958',
  'sugai',
  '',
  ''],
 '3': ['techniqu',
  'depart',
  'matrix',
  'program',
  'scheme',
  'cacm',
  'decemb',
  '',
  '1958',
  'friedman',
  '',
  ''],
 '4': ['glossari',
  'comput',
  'engin',
  'program',
  'terminolog',
  'cacm',
  'novemb',
  '',
  '1958'],
 '5': ['squareroot',
  'approxim',
  'cacm',
  'novemb',
  '',
  '1958',
  'wadey',
  '',
  ''],
 '6': ['comput',
  'inspect',
  'procedur',
  'cacm',
  'novemb',
  '',
  '1958',
  'muller',
  '',
  ''],
 '7': ['glossari',
  'comput',
  'engin',
  'program',
  'terminolog',
  'cacm',
  'octob',
  '',
  '1958'],
 '8': ['equival',
  'transform',
  'program',
  'scheme',
  'cacm',
  'octob',
  '',
  '1958',
  'friedman',
  '',
  ''],
 '9': ['propos', 'uncol', 'cacm', '

In [14]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatize_list=[]
for store,x in chunked_corpus.items():
    for i in corpus_dict[store]:
        lemmatize_list.append(lemmatizer.lemmatize(i))
    corpus_dict[store]=lemmatize_list
    lemmatize_list=[]
corpus_dict    

{'1': ['preliminari',
  'reportintern',
  'algebra',
  'languag',
  'cacm',
  'decemb',
  '',
  '1958',
  'perli',
  '',
  'samelson',
  '',
  ''],
 '2': ['extract',
  'root',
  'repeat',
  'subtract',
  'digit',
  'comput',
  'cacm',
  'decemb',
  '',
  '1958',
  'sugai',
  '',
  ''],
 '3': ['techniqu',
  'depart',
  'matrix',
  'program',
  'scheme',
  'cacm',
  'decemb',
  '',
  '1958',
  'friedman',
  '',
  ''],
 '4': ['glossari',
  'comput',
  'engin',
  'program',
  'terminolog',
  'cacm',
  'novemb',
  '',
  '1958'],
 '5': ['squareroot',
  'approxim',
  'cacm',
  'novemb',
  '',
  '1958',
  'wadey',
  '',
  ''],
 '6': ['comput',
  'inspect',
  'procedur',
  'cacm',
  'novemb',
  '',
  '1958',
  'muller',
  '',
  ''],
 '7': ['glossari',
  'comput',
  'engin',
  'program',
  'terminolog',
  'cacm',
  'octob',
  '',
  '1958'],
 '8': ['equival',
  'transform',
  'program',
  'scheme',
  'cacm',
  'octob',
  '',
  '1958',
  'friedman',
  '',
  ''],
 '9': ['propos', 'uncol', 'cacm', '

In [15]:
z="I am interested in articles written either by Prieve or Udo Pooch and i love my home"

In [16]:
z

'I am interested in articles written either by Prieve or Udo Pooch and i love my home'

In [17]:
def remove_common_words_tok_lower(query):
    filtered= []
    for w in word_tokenize(query.lower()):
        if w not in common_words:
            filtered.append(w)
    return filtered

In [18]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmtization_query(query):
    lemmatized_string= [lemmatizer.lemmatize(words) for words in query]
    return lemmatized_string

In [19]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def stemming_query(query):
    stemm =[ps.stem(words) for words in query]
    return stemm
    

In [20]:
z = remove_common_words_tok_lower(z)

In [21]:
z

['interested', 'articles', 'written', 'prieve', 'udo', 'pooch', 'love', 'home']

In [22]:
z=stemming_query(z)
z=stemming_query(z)

In [23]:
query_text=[]
string1=''
for i in z:
    string1=string1+str(i)+' '
query_text.append(string1)
    

In [24]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

In [25]:
txt_doc=''
txt_index=[]
for store,x in corpus_dict.items():
    for i in x:
        txt_doc=txt_doc+i+' '
    txt_index.append(txt_doc) 
    txt_doc=''
len(txt_index)

3204

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [27]:
tfidf_vectorizer = TfidfVectorizer()

In [28]:
tfidf_matrix_train = tfidf_vectorizer.fit_transform(txt_index)


In [29]:
tfidf_matrix_test = tfidf_vectorizer.transform(query_text)


In [30]:
cos=cosine_similarity(tfidf_matrix_train,tfidf_matrix_test)

In [30]:
cos

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.05357365],
       [0.        ],
       [0.03068536]])

In [36]:
s={}
h=1
for i in cos:
    for j in i:
        s[h]=j
    h=h+1
res=dict(sorted(s.items(),key = itemgetter(1),reverse=True)[:10])
res

{823: 0.19226686602436824,
 136: 0.1079184567119884,
 740: 0.10249567708241338,
 356: 0.10212945008741026,
 2918: 0.10088526889302442,
 597: 0.082570844780848,
 1345: 0.08218909956527198,
 971: 0.06961354903650577,
 1046: 0.069318512999767,
 1755: 0.068159919588057}

In [37]:

for i in res:
    print(i)
        

823
136
740
356
2918
597
1345
971
1046
1755


In [38]:
from sklearn.metrics import classification_report
from sklearn import metrics

In [39]:
from pandas import DataFrame
df=DataFrame(tfidf_matrix_train.toarray(), 
                     columns=tfidf_vectorizer.get_feature_names_out())
(df)

,01,034,10,100,10000,1000000,100200,100fold,101,102,...,zobrist,zoeren,zone,zooplankton,zuckerman,zund,zuse,zwakenberg,zweben,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#evaluation

In [41]:
qrels = open("/Users/Ahmad98/Desktop/IR_Proj/cacm/qrels.text","r")

In [70]:
for i in res:
    print(i)

823
136
740
356
2918
597
1345
971
1046
1755


In [35]:
qre=pd.read_csv("/Users/Ahmad98/Desktop/IR_Proj/cacm/qrels.text")

In [36]:
qre

,01 1410 0 0
0,01 1572 0 0
1,01 1605 0 0
2,01 2020 0 0
3,01 2358 0 0
4,02 2434 0 0
...,...
790,63 2714 0 0
791,63 2973 0 0
792,63 3075 0 0
793,63 3156 0 0


In [33]:
def calculate_prec(res, gold_standard):
    true_pos = 0
    for item,x in res.items():
        for i,j in mappings.items():
            if i=='01':
                for k in j:
                    
                    if k==str(item):
                        print(k)
                        true_pos+=1    
    print(true_pos)        
    return float(true_pos)/float(len(res.items()))



In [34]:
def calculate_recal(res, gold_standard):
    true_pos = 0
    for item,x in res.items():
        for i,j in mappings.items():
            if i=='01':
                x=len(j)
                for k in j:
                    
                    if k==str(item):
                        print(k)
                        true_pos+=1    
    print(true_pos)        
    return float(true_pos)/float(x)

In [42]:
calculate_prec(res,qrels)

NameError: name 'mappings' is not defined

In [ ]:
## Another way

In [99]:
df['y_act']=(df.comput>0.1).astype('int')
df['y_pred_rf']=(df.algebra>=0.2).astype('int')
df['y_pred_lr']=(df.program>=0.2).astype('int')

In [100]:
df

,01,034,10,100,10000,1000000,100200,100fold,101,102,...,zooplankton,zuckerman,zund,zuse,zwakenberg,zweben,zz,y_act,y_pred_rf,y_pred_lr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0
3202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0


In [101]:
def compute_tp_tn_fn_fp(y_act,y_pred):
    tp=sum((y_act==1)&(y_pred==1))
    tn=sum((y_act==0)&(y_pred==0))    
    fn=sum((y_act==1)&(y_pred==0))    
    fp=sum((y_act==0)&(y_pred==1))
    return tp,tn,fp,fn

In [102]:
tp_rf,tn_rf,fp_rf,fn_rf=compute_tp_tn_fn_fp(df.y_act,df.y_pred_lr)
print('TP for program : ',tp_rf)
print('TN for program : ',tn_rf)
print('FP for program : ',fp_rf)
print('FN for program : ',fn_rf)

TP for program :  22
TN for program :  2745
FP for program :  87
FN for program :  350


In [103]:
tp_lr,tn_lr,fp_lr,fn_lr=compute_tp_tn_fn_fp(df.y_act,df.y_pred_rf)
print('TP for algebra : ',tp_lr)
print('TN for algebra : ',tn_lr)
print('FP for algebra : ',fp_lr)
print('FN for algebra : ',fn_lr)

TP for algebra :  5
TN for algebra :  2806
FP for algebra :  26
FN for algebra :  367


In [104]:
from sklearn.metrics import confusion_matrix
tn_rf1,fp_rf1,fn_rf1,tp_rf1=confusion_matrix(df.y_act,df.y_pred_rf).ravel()
print('TP for program : ',tp_rf1)
print('TN for program : ',tn_rf1)
print('FP for program : ',fp_rf1)
print('FN for program : ',fn_rf1)

TP for program :  5
TN for program :  2806
FP for program :  26
FN for program :  367


In [105]:
def compute_accuracy(tp, tn, fn, fp):

    return ((tp + tn) * 100)/ float( tp + tn + fn + fp)

In [106]:
print('Accuracy for program :', compute_accuracy(tp_lr, tn_lr, fn_lr, fp_lr))
print('Accuracy for algebra :', compute_accuracy(tp_rf, tn_rf, fn_rf, fp_rf))

Accuracy for program : 87.73408239700375
Accuracy for algebra : 86.36079900124844


In [47]:
#from sklearn.metrics import accuracy_score

#print('Accuracy for program :', 100* accuracy_score(df.y_act df.y_pred_lr))
#print('Accuracy for program :', 100*accuracy_score(df.y_act,  df.y_pred_rf))

In [48]:
def compute_precision(tp, fp):

    return (tp  * 100)/ float( tp + fp)

In [107]:
print('Precision for program :', compute_precision(tp_lr, fp_lr))
print('Precision for algebra :', compute_precision(tp_rf, fp_rf))

Precision for program : 16.129032258064516
Precision for algebra : 20.18348623853211


In [50]:
#from sklearn.metrics import precision_score
#print('Precision for program :', 100* precision_score(df.y_act, df.y_pred_lr))
#print('Precision for algebra :', 100* precision_score(df.y_act,df.y_pred_rf))

In [108]:
def compute_recall(tp, fn):

    return (tp  * 100)/ float( tp + fn)

In [109]:
print('Recall for program :', compute_recall(tp_lr, fn_lr))
print('Recall for algebra :', compute_recall(tp_rf, fn_rf))

Recall for program : 1.3440860215053763
Recall for algebra : 5.913978494623656


In [53]:
#from sklearn.metrics import recall_score

#print('Recall for program :', 100* recall_score(df.y_act,df.y_pred_lr))
#print('Recall for algebra :', 100* recall_score(df.y_act, df.y_pred_rf))

In [110]:
def compute_f1_score(y_true, y_pred):
    tp, tn, fp, fn = compute_tp_tn_fn_fp(y_true, y_pred)
    precision = compute_precision(tp, fp)/100
    recall = compute_recall(tp, fn)/100
    f1_score = (2*precision*recall)/ (precision + recall)
    return f1_score

In [111]:
print('F1 score for program :', compute_f1_score(df.y_act, df.y_pred_lr))
print('F1 score for algebra :', compute_f1_score(df.y_act,  df.y_pred_rf))

F1 score for program : 0.09147609147609148
F1 score for algebra : 0.024813895781637715


In [56]:
#from sklearn.metrics import f1_score
#print('F1 score for program :', f1_score(df.y_act, df.y_pred_lr))
#print('F1 score for algebra :', f1_score(df.y_act, df.y_pred_rf))